In [3]:
from pymodbus.client import ModbusSerialClient
import struct
import time

# --- CONFIGURATION ---
PORT = 'COM5'          # Replace with your COM port, e.g., '/dev/ttyUSB0' on Linux
BAUDRATE = 9600
PARITY = 'E'           # Usually 'E' for Even
STOPBITS = 1
BYTESIZE = 8
SLAVE_ID = 1           # Modbus address of Omron E5CC

# --- CONNECT TO MODBUS RTU ---
client = ModbusSerialClient(
    port=PORT,
    baudrate=BAUDRATE,
    parity=PARITY,
    stopbits=STOPBITS,
    bytesize=BYTESIZE,
    timeout=1
)

if not client.connect():
    print("❌ Failed to connect to Modbus device.")
    exit()

def read_float32(address):
    """Read a 4-byte (32-bit) float from Modbus."""
    rr = client.read_holding_registers(address=address, count=2, slave=SLAVE_ID)
    if rr.isError():
        print(f"⚠️ Error reading address {address}: {rr}")
        return None
    
    regs = rr.registers
    # Try Big Endian first
    packed = struct.pack('>HH', regs[0], regs[1])
    value = struct.unpack('>f', packed)[0]
    return value

try:
    while True:
        # PV (Process Value) - Address 2000h
        pv = read_float32(0x2402)
        if pv is not None:
            print(f"🌡️ Process Value (PV): {pv:.2f} °C")

        # SV (Set Value) - Address 2002h
        sv = read_float32(0x2403)
        if sv is not None:
            print(f"🎯 Set Value (SV): {sv:.2f} °C")

        time.sleep(2)

except KeyboardInterrupt:
    print("\n🛑 Stopped by user.")

finally:
    client.close()


TypeError: ModbusClientMixin.read_holding_registers() got an unexpected keyword argument 'slave'

In [5]:
# Compatibility Modbus RTU reader for Omron E5CC (handles pymodbus v2.x & v3.x)
import struct
import time

# ---------- USER CONFIG ----------
PORT = 'COM5'          # e.g., '/dev/ttyUSB0' on Linux
BAUDRATE = 9600
PARITY = 'E'
STOPBITS = 1
BYTESIZE = 8
SLAVE_ID = 1           # Modbus address of your E5CC
TIMEOUT = 1
# ---------------------------------

# try to import pymodbus and show version if available
pymodbus_version = None
try:
    import pymodbus
    try:
        pymodbus_version = pymodbus.__version__
    except Exception:
        pymodbus_version = "unknown"
except ImportError:
    pymodbus = None
    pymodbus_version = None

print("pymodbus module present:", pymodbus is not None, "version:", pymodbus_version)

# Try multiple import paths for ModbusSerialClient (v3 vs v2)
ModbusSerialClient = None
client = None

# Try v3 import path first
try:
    from pymodbus.client import ModbusSerialClient as MSC_v3
    ModbusSerialClient = MSC_v3
    print("Using pymodbus.client.ModbusSerialClient (likely v3)")
except Exception:
    try:
        # v2 import path
        from pymodbus.client.sync import ModbusSerialClient as MSC_v2
        ModbusSerialClient = MSC_v2
        print("Using pymodbus.client.sync.ModbusSerialClient (likely v2)")
    except Exception as e:
        print("Failed to import ModbusSerialClient:", e)
        raise

# Instantiate client trying both constructor signatures (with/without method)
client = None
construct_errors = []
try:
    # v2 expects method='rtu'
    client = ModbusSerialClient(method='rtu',
                                port=PORT,
                                baudrate=BAUDRATE,
                                parity=PARITY,
                                stopbits=STOPBITS,
                                bytesize=BYTESIZE,
                                timeout=TIMEOUT)
    print("Client created with method='rtu' constructor.")
except TypeError as e:
    construct_errors.append(("method_ctor", str(e)))
    try:
        # v3 style – omit method
        client = ModbusSerialClient(port=PORT,
                                    baudrate=BAUDRATE,
                                    parity=PARITY,
                                    stopbits=STOPBITS,
                                    bytesize=BYTESIZE,
                                    timeout=TIMEOUT)
        print("Client created with no-method constructor.")
    except Exception as e2:
        construct_errors.append(("no_method_ctor", str(e2)))
        print("Client construction failures:", construct_errors)
        raise

# Connect
if not client.connect():
    print("❌ Failed to connect to Modbus device. Check port and wiring.")
else:
    print("✅ Connected to Modbus device.")

# Robust wrapper to call read_holding_registers using multiple argument styles
def _call_read(address, count):
    """
    Try calling client.read_holding_registers with several argument styles
    to support different pymodbus versions.
    Returns the raw result object or raises last exception.
    """
    last_exc = None
    try:
        # try v3 style
        return client.read_holding_registers(address=address, count=count, slave=SLAVE_ID)
    except TypeError as e:
        last_exc = e
    except Exception as e:
        # may still be valid (e.g., connection error) - return to caller
        last_exc = e

    try:
        # try v2 style 'unit'
        return client.read_holding_registers(address=address, count=count, unit=SLAVE_ID)
    except TypeError as e:
        last_exc = e
    except Exception as e:
        last_exc = e

    try:
        # try positional: (address, count, unit)
        return client.read_holding_registers(address, count, SLAVE_ID)
    except TypeError as e:
        last_exc = e
    except Exception as e:
        last_exc = e

    try:
        # try without unit/slave (device might be the only one on bus)
        return client.read_holding_registers(address, count)
    except Exception as e:
        last_exc = e

    # if none worked, raise last exception for debugging
    raise last_exc

def read_float32(address, debug=False):
    """Read two registers (4 bytes) and return float (tries both endian orders)."""
    try:
        rr = _call_read(address=address, count=2)
    except Exception as e:
        print(f"❗ Read failed for addr {hex(address)}: {e}")
        return None

    # rr may be None or indicate error
    if rr is None:
        print(f"❗ No response object for address {hex(address)}")
        return None

    # different pymodbus versions use different ways to signal errors
    # check common patterns:
    if hasattr(rr, 'isError') and rr.isError():
        print(f"⚠️ Modbus returned error object for {hex(address)}: {rr}")
        return None

    if not hasattr(rr, 'registers'):
        # some implementations put registers under .registers or .getRegister
        print("⚠️ Response has no .registers attribute; raw response:", rr)
        return None

    regs = rr.registers
    if len(regs) < 2:
        print(f"⚠️ Insufficient registers length ({len(regs)}) for 4-byte value at {hex(address)}")
        return None

    if debug:
        print(f"DEBUG registers @ {hex(address)}:", regs)

    # Try big-endian pair -> float
    try:
        packed_be = struct.pack('>HH', regs[0] & 0xFFFF, regs[1] & 0xFFFF)
        val_be = struct.unpack('>f', packed_be)[0]
    except Exception as e:
        val_be = None

    # Try little-endian swap -> float
    try:
        packed_le = struct.pack('<HH', regs[1] & 0xFFFF, regs[0] & 0xFFFF)
        val_le = struct.unpack('<f', packed_le)[0]
    except Exception as e:
        val_le = None

    # Heuristics: choose the value that looks reasonable
    def plausible(v):
        if v is None: 
            return False
        # reject NaN/inf and extreme nonsense
        if v != v: 
            return False
        if abs(v) > 1e6: 
            return False
        return True

    if plausible(val_be) and not plausible(val_le):
        chosen = ('BE', val_be)
    elif plausible(val_le) and not plausible(val_be):
        chosen = ('LE', val_le)
    elif plausible(val_be) and plausible(val_le):
        # both plausible — prefer BE (common for Omron), but show both
        chosen = ('BE', val_be)
    else:
        # neither plausible — still return BE as fallback and show both
        chosen = ('BE', val_be)

    if debug:
        print(f"decoded BE={val_be}, LE={val_le} -> chosen {chosen[0]} = {chosen[1]}")

    return chosen[1]

# Example usage: read the address you tried earlier (0x2402)
try:
    for _ in range(3):
        pv = read_float32(0x2402, debug=True)  # debug=True shows registers & both decodes
        if pv is not None:
            print(f"PV @ 0x2402 -> {pv:.4f}")
        else:
            print("No valid PV value read.")
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped by user.")
finally:
    try:
        client.close()
    except Exception:
        pass


pymodbus module present: True version: 3.11.3
Using pymodbus.client.ModbusSerialClient (likely v3)
Client created with no-method constructor.
✅ Connected to Modbus device.
❗ Read failed for addr 0x2402: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2402: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2402: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.


In [7]:
from pymodbus.client import ModbusSerialClient
import struct
import time

# --- CONFIGURATION ---
PORT = "COM5"        # Replace with your serial port
BAUDRATE = 9600
PARITY = 'N'
STOPBITS = 1
BYTESIZE = 8
SLAVE_ID = 1          # Modbus address of your E5CC

# --- Create Client ---
client = ModbusSerialClient(
    port=PORT,
    baudrate=BAUDRATE,
    parity=PARITY,
    stopbits=STOPBITS,
    bytesize=BYTESIZE,
    timeout=1
)

if client.connect():
    print("✅ Connected to Modbus device.")
else:
    print("❌ Failed to connect.")
    exit()

# --- Function to Read 32-bit Float (4 bytes) ---
def read_float32(address):
    """Read a 4-byte (32-bit) float from Modbus register (two consecutive 16-bit)."""
    try:
        # pymodbus v3.11.3 expects positional arguments only
        rr = client.read_holding_registers(address, 2)
        if rr.isError():
            print(f"⚠️ Modbus error at {hex(address)}: {rr}")
            return None
        registers = rr.registers
        if len(registers) != 2:
            print("⚠️ Unexpected register length.")
            return None
        # Combine two 16-bit registers → one 32-bit float (big endian)
        raw = struct.pack('>HH', registers[0], registers[1])
        value = struct.unpack('>f', raw)[0]
        return value
    except Exception as e:
        print(f"❗ Read failed for addr {hex(address)}: {e}")
        return None

# --- Main Loop ---
try:
    while True:
        pv = read_float32(0x2000)   # E5CC PV register address
        if pv is not None:
            print(f"🌡️ Process Value (PV): {pv:.2f} °C")
        else:
            print("No valid PV value read.")
        time.sleep(1)

except KeyboardInterrupt:
    print("🛑 Stopped by user.")
finally:
    client.close()


✅ Connected to Modbus device.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but 3 were given
No valid PV value read.
❗ Read failed for addr 0x2000: ModbusClientMixin.read_holding_registers() takes 2 positional arguments but